In [15]:
import pandas as pd
import pickle
from utils import encode
sep = '|'

In [4]:

df_orig = pd.read_csv("..\\raw_csv\\2022_Annual_Survey_Aggregate_Data_for_2022_Annual_Survey_20220905222712-SurveyExport.csv", sep = sep)
is_military = df_orig['Are you, or were you ever, an active member of the military or member of the National Guard or Reserve?'] == 'Yes'
df_orig_military = df_orig[is_military].copy().reset_index()
df_orig_civ = df_orig[~is_military]
df_orig_military['All'] = 'All'

In [12]:
with open("Encodings_2022.dat", 'rb') as f:
    encodings = pickle.load(f)

In [ ]:
dfAdapt = encode(df_orig_military[[x for x in df_orig_military.columns if any(y in x.lower() for y in ['adapt ', 'bounce', 'hope scale score'])]], conversion_dictionary).dropna()
dfAdapt_corr = dfAdapt.corr().sort_values('Hope Scale Score', ascending = False)
dfAdapt_corr2 = dfAdapt_corr['Hope Scale Score'].reset_index().dropna()[1:]
dfAdapt_corr2['Hope Scale Score'] = dfAdapt_corr2['Hope Scale Score'].apply(round, args = [3])
dfAdapt_corr2.columns = ['Adapt Question', 'Hope Scale Score Correlation']
dfAdapt_corr2.set_index('Adapt Question')

In [ ]:
dfHope =encode(df_orig_military[[x for x in df_orig_military if 'hope' in x.lower()]], conversion_dictionary).dropna()

dfHope_corr.index = [x.split(':')[0] for x in dfHope_corr.index]
dfHope_corr2 = dfHope_corr['Hope Scale Score'].reset_index().dropna()[1:]
dfHope_corr2['Hope Scale Score'] = dfHope_corr2['Hope Scale Score'].apply(round, args = [3])
dfHope_corr2.columns = ['Hope Question', 'Hope Scale Score Correlation']
dfHope_corr2.set_index('Hope Question')

In [ ]:
dfAgree = df_orig_military[[x for x in df_orig_military if 'agree' in x.lower() or x.lower() == 'hope scale score']]
dfAgree = encode(dfAgree, conversion_dictionary)
dfAgree = dfAgree[~dfAgree['Hope Scale Score'].isna()]
dfAgree = dfAgree.dropna()
dfAgree = dfAgree.replace(r'?', np.nan)
uns = set()
for col in dfAgree:
    un = dfAgree[col].unique()
    for x in un:
        if isinstance(x, str):
            uns.add(x)
    num_na = dfAgree[col].isna().sum()
    if num_na == len(dfAgree):
        dfAgree = dfAgree.drop(columns = col)
for col in dfAgree:
    
    dfAgree[col] = dfAgree[col].apply(lambda x: np.nan if x in uns else x)
    dfAgree[col] = dfAgree[col].fillna(dfAgree[col].mean())

dfAgree_corr = dfAgree.corr().sort_values('Hope Scale Score', ascending = False)
dfAgree_corr.index = [x.split(':')[0] for x in dfAgree_corr.index]
dfAgree_corr2 = dfAgree_corr['Hope Scale Score'].reset_index().dropna()[1:]
dfAgree_corr2['Hope Scale Score'] = dfAgree_corr2['Hope Scale Score'].apply(round, args = [3])
dfAgree_corr2.columns = ['Agree/Disagree Question', 'Hope Scale Score Correlation']
dfAgree_corr2.set_index('Agree/Disagree Question')

In [ ]:
dfSeek = df_orig_military[[x for x in df_orig_military if (('seeking' in x.lower() and not 
                                                            'you mentioned you are unemployed' in x.lower())
                                                           or x.lower() == 'hope scale score')]]
dfSeek = encode(dfSeek, conversion_dictionary)
dfSeek = dfSeek[~dfSeek['Hope Scale Score'].isna()]
#dfSeek = dfSeek.dropna()
replace_dict = {}
for col in dfSeek:
    if col == 'Hope Scale Score':
        continue
    uns = dfSeek[col].unique()
    for un in uns:
        if not un in replace_dict:
            print(uns)
            print("WHAT IS :", un)
            replace_dict[un] = float(input())
    dfSeek[col] = dfSeek[col].apply(replace_dict.__getitem__)
dfSeek = dfSeek[dfSeek['Hope Scale Score'] < mn - stdev]
dfSeek_corr = dfSeek.corr().sort_values('Hope Scale Score', ascending = False)
dfSeek_corr.index = [x.split(':')[0] for x in dfSeek_corr.index]
dfSeek_corr2 = dfSeek_corr['Hope Scale Score'].reset_index().dropna()[1:]
dfSeek_corr2['Hope Scale Score'] = dfSeek_corr2['Hope Scale Score'].apply(round, args = [3])
dfSeek_corr2.columns = ['Agree/Disagree Question', 'Hope Scale Score Correlation']
dfSeek_corr2.set_index('Agree/Disagree Question')

In [25]:
dfDistance = df_orig_military[['How far is the Veterans Affairs (VA) medical facility within in miles of where you live? Â\xa0',
                          'Hope Scale Score']]
dfDistance = dfDistance[~dfDistance['Hope Scale Score'].isna()]
dfDistance = encode(dfDistance, encodings)
dfDistance_corr = dfDistance.corr().sort_values('Hope Scale Score', ascending = False)
dfDistance_corr2 = dfDistance_corr['Hope Scale Score'].reset_index().dropna()[1:]
dfDistance_corr2['Hope Scale Score'] = dfDistance_corr2['Hope Scale Score'].apply(round, args = [3])
dfDistance_corr2.columns = ['', 'Hope Scale Score Correlation']
dfDistance_corr2.set_index('')

,Hope Scale Score Correlation
,
How far is the Veterans Affairs (VA) medical facility within in miles of where you live? Â,0.017


In [22]:
dfDistance_corr2

,index,Hope Scale Score
1,How far is the Veterans Affairs (VA) medical f...,0.017


In [9]:
df_orig_military[['How far is the Veterans Affairs (VA) medical facility within in miles of where you live? Â\xa0',
                          'Hope Scale Score']]

,How far is the Veterans Affairs (VA) medical facility within in miles of where you live? Â,Hope Scale Score
0,20 miles or more,59.0
1,Less than 10 miles away,14.0
2,20 miles or more,43.0
3,20 miles or more,50.0
4,20 miles or more,42.0
...,...,...
1366,20 miles or more,57.0
1367,Less than 10 miles away,49.0
1368,10 miles,24.0
1369,20 miles or more,62.0
